In [ ]:
import polars as pl
import seaborn as sb
import gtfs_delay_analysis as da

In [ ]:
# 2002
import importlib
importlib.reload(da.ddelay)

In [ ]:
aggregated = da.load_aggregate_data()
stops = pl.read_parquet('stops.parquet')
raw_dfs = da.load_raw_data()

In [ ]:
trips = da.trips.load_trips_without_shapes_df()
str_shapes = da.trips.load_str_shapes_df()
shapes = da.trips.load_parsed_shapes_df()
sequences = pl.read_parquet('stop-sequences.parquet')

In [ ]:
all_004 =aggregated.join(trips, left_on='id', right_on='trip_id').filter(
    pl.col("shape_id").eq("004-171-East")
)

# Delay
# Derivative of delay

In [ ]:
df = (
all_004.with_columns(pl.col('stopid').cast(pl.Utf8))
.join(
    sequences.with_columns(pl.col('stop_id').cast(pl.Utf8)),
    left_on=['id', 'stopid'],
    right_on=['trip_id', 'stop_id'],
)
.group_by('id', 'stopid', 'stop_sequence').agg(
    pl.col('meandelay').mean(),
)
.sort('id', 'stop_sequence')
)

specific_trip = df.filter(pl.col('id').eq(25536739))

In [ ]:
def reorder_stops(df:pl.DataFrame, stop_id:str):
    stop_idx =df.filter(pl.col('stopid').eq(stop_id))['stop_sequence'][0]
    max_idx=df['stop_sequence'].max()
    df['stop_sequence'].max()
    return df.with_columns(
        pl.col('stop_sequence').sub(stop_idx).mod(max_idx)
    ).sort('stop_sequence')




In [ ]:
def add_stops(df: pl.DataFrame):
    return (
        df
    .with_row_index()
        .with_columns(pl.col('line').str.split('-'))
    .with_columns(
        pl.col('line').list.get(0).alias('a'),
        pl.col('line').list.get(1).alias('b'),
    )
    .drop('line')
    .join(stops.select('stop_id', a_lon='stop_lon', a_lat='stop_lat'), left_on='a', right_on='stop_id')
    .join(stops.select('stop_id', b_lon='stop_lon', b_lat='stop_lat'), left_on='b', right_on='stop_id')
    .sort('index')
    .drop('index')
    )

In [ ]:
from gtfs_delay_analysis.ddelay import get_ddelay,plot_ddelay

def ddelay_with_seq(d:pl.DataFrame):
    return (
        d.join(stops, left_on='stopid',right_on='stop_id')
        .with_columns( trip_id='id')
        .sort('stop_sequence')
        # .pipe(lambda d: display(d) or d)
        .with_row_index()
        .pipe(get_ddelay)
    )

def save(df:pl.DataFrame,name:str):
    import os
    write_path ='~/Downloads/GTFS'
    return df.write_csv(os.path.join(write_path, name))

def pre_process(df:pl.DataFrame):
    return df .pipe(ddelay_with_seq) .drop('trip_id') .pipe(add_stops) .with_row_index()

with_coords =(
df.group_by('stopid', 'stop_sequence')
.agg( pl.col('meandelay').mean())
.with_columns(id=0)
.pipe(reorder_stops, "2002")
.pipe(pre_process)
# .pipe(plot_ddelay)
# .pipe(save,'ddelay-all-4.csv')
)

# specific_trip.pipe(reorder_stops, "2002").pipe(ddelay_with_seq).pipe(save, 'ddelay-25536739.csv')
all_points = shapes.filter(pl.col('shape_id').eq('004-171-East')).explode('geometry_line').rename({
    'geometry_line':'points',
}).with_row_index()

# with_coords
points = all_points.select(
    'index','shape_id',
    lon=pl.col('points').struct.field('lon'),
    lat=pl.col('points').struct.field('lat'),
)


In [ ]:
(df.group_by('stopid', 'stop_sequence')
.agg( pl.col('meandelay').mean())
.with_columns(id=0)
.pipe(reorder_stops, "2002")
).pipe(save, 'heading-stops.csv')

In [ ]:
pp = points.with_columns(
    pl.col('index').sub(1).alias('prev')
).join(points,left_on='prev',right_on='index').select(
    index='prev',
    a_lon='lon',
    a_lat='lat',
    b_lon='lon_right',
    b_lat='lat_right',
)
# .pipe(save ,'fuck-shapes.csv')
with_coords.head(12).pipe(save, 'coords.csv')

In [ ]:
str_shapes.filter(pl.col('shape_id').eq('004-171-East')).pipe(save ,'actual-shape.csv')

In [ ]:
specific_trip.join(stops, left_on='stopid',right_on='stop_id').sort('stop_sequence').with_columns(
    trip_id='id'
).with_row_index()

In [ ]:
shapes.filter(pl.col('shape_id').str.starts_with('004-171').or_(
pl.col('shape_id').str.starts_with('004-172')
)).explode('geometry_line').select(
    'shape_id',
    lon=pl.col('geometry_line').struct.field('lon'),
    lat=pl.col('geometry_line').struct.field('lat'),
).write_csv('~/Downloads/GTFS/004-East-shapes.csv')

In [ ]:
from geopy.distance import geodesic

In [ ]:
def calculate_slope(df:pl.DataFrame):
    return df.with_columns(
        pl.struct(
            lon1='a_lon',
            lat1='a_lat',
            lon2='b_lon',
            lat2='b_lat',
        ).map_elements(lambda x: geodesic((x['lat1'], x['lon1']), (x['lat2'], x['lon2'])).km).alias('distance')
    ).with_columns(pl.col('ddelay').truediv('distance').alias('slope'))

# with_coords.head(12).pipe(calculate_slope).pipe(save,'heading-slope.csv')
specific_trip.pipe(reorder_stops, "2002").pipe(save,'specific-stops.csv')
# .pipe(pre_process).head(12).pipe(calculate_slope).pipe(save,'specific-trip-slope.csv')

In [ ]:
import seaborn as sb

In [ ]:
from io import StringIO
df = pl.read_csv(StringIO("""distance,meandelay,ddelay
0.260948712,780,-132
0.49200577,648,123
0.762336063,771,-107
1.105268809,664,86
1.36051648,750,9
1.696549156,759,-58.5
2.516094026,700.5,122.8333333
2.685788091,823.3333333,-6.666666667
3.067528361,816.6666667,26.66666667
3.451850377,843.3333333,-71.66666667
3.671237928,771.6666667,31.66666667
4.095030012,803.3333333,36.66666667"""))

In [ ]:
ax =sb.lineplot(df, x='distance', y='ddelay',drawstyle='steps-pre')

In [ ]:
df